In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import deepmatcher as dm
import random as rd
from utils.dataset_parser import generate_unlabeled
from similarity.jaccard import Jaccard

In [7]:
tableA = pd.read_csv('Structured/Amazon-Google/tableA.csv')
tableB = pd.read_csv('Structured/Amazon-Google/tableB.csv')

In [6]:
def filter_fn(row,comparator,att,val,threshold):
    if comparator.similarity(row[att],val) >=threshold:
        return False
    else:
        return True

In [7]:
##the attributes are in order of importance
def createPossibleMatchings(df1,df2,important_attributes,min_similarity,newsample_len):
    comparator = Jaccard(3)
    df1_index = list(df1.index)
    df2_index = list(df2.index)
    newSamples = []
    while(len(newSamples)<newsample_len):
        current_lrow = df1.iloc[rd.choice(df1.index)]
        current_df = df2
        for att in important_attributes:
            l_att = str(current_lrow[att])
            mask = current_df.apply(lambda row: comparator.similarity(str(row[att]),l_att)
                                    >=min_similarity,axis=1)
            current_df = current_df[mask]
        for idx in current_df.index:
            newSamples.append((current_lrow['id'],current_df.at[idx,'id']))
    return newSamples

In [18]:
newSamples = createPossibleMatchings(tableA,tableB,['manufacturer','title'],0.4,20)

In [21]:
newSamples_ids = pd.DataFrame(data=newSamples,columns=['ltable_id','rtable_id'])
newSamples_ids.to_csv('Structured/Amazon-Google/unlabeled_ids.csv',index=False)
newSamples_df = generate_unlabeled('Structured/Amazon-Google/','unlabeled_ids.csv')

In [22]:
newSamples_df.head(10)

,ltable_title,ltable_manufacturer,ltable_price,rtable_title,rtable_manufacturer,rtable_price,id
0,instant immersion language lab deluxe edition,topics entertainment,NaN,instant immersion 33 languages,topics entertainment,43.99,0
1,creating keepsakes scrapbook designer version ...,riverdeep-learning-company,39.99,scrapbook designer platinum,riverdeep-learning-company,17.90,1
2,studyworks ! teaching pro : middle school math...,global-software-publishing,29.99,middle school math & science,global-software-publishing,12.90,2
3,studyworks ! teaching pro : middle school math...,global-software-publishing,29.99,middle school math & science,global-software-publishing,12.90,3
4,quicken 2007 premier,intuit,79.95,quicken 2007 premier win cd,intuit,73.99,4
5,internet cleanup 4.0 mac os 10.4 or higher,smith-micro-software,29.99,allume internet cleanup 4.0,smith-micro-software,29.99,5
6,instant immersion italian deluxe v2 .0 ( large...,topics entertainment,29.99,instant immersion italian 2.0,topics entertainment,20.99,6
7,totally spies : zombie jamboree ( win/mac ),brighter-minds-media-inc .,9.95,totally spies zombie jamboree by brighter minds,brighter-minds-media-inc .,9.99,7
8,instant immersion spanish audio deluxe,topics entertainment,39.95,topics entertainment 40248 instant immersion s...,topics entertainment,23.61,8
9,ibm viavoice advanced v. 10,nuance-communications-inc .,NaN,nuance communications inc. ibm viavoice advanc...,nuance-communications-inc .,57.54,9


In [23]:
newSamples_df.to_csv('amazon_google_unlabeled.csv',index=False)

In [32]:
model = dm.MatchingModel(attr_summarizer='hybrid')
model.load_state('models/itunes_amazon_hybrid.pth')

In [34]:
unlabeled_dm = dm.data.process_unlabeled('itunes_amazon_unlabeled.csv',model,ignore_columns=['id'])
predictions = model.run_prediction(unlabeled_dm,output_attributes=True)


Reading and processing data from "itunes_amazon_unlabeled.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 6
Finished Epoch 6 || Run Time:    0.4 | Load Time:    0.2 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [38]:
matching_predictions = predictions[predictions.match_score <=0.5]
matching_predictions.head()

,match_score,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
id,,,,,,,,,,,,,,,,,
3,0.358066,Hourglass ( feat . Jessica Sanchez ) [ iTunes ...,Zedd,iTunes Session - EP,"Dance , Music",$ 1.29,2013 Interscope Records,3:42,19-Nov-13,Look At Me Now ( Clean Version ) [ Clean ],Alyxx Dione feat . Jason Derulo,Chris Brown and More,Pop,$ 1.29,( C ) 2008 Chris Young,3:42,NaN
5,0.162936,Thriller ( Live From Hammersmith Palais ),Fall Out Boy,Infinity On High - Deluxe Bonus - EP,"Alternative , Music , Punk , Rock , Adult Alte...",$ 0.99,2007 The Island Def Jam Music Group,3:32,20-Nov-07,Adrenaline [ Clean ],Blake Shelton,Bass Drops & Fist-Pumps,Country,$ 0.89,( c ) 2004 Los Cangris Music,3:32,NaN
9,0.357007,The Edge of Glory ( Live from `` A Very Gaga T...,Lady Gaga,A Very Gaga Holiday ( Live ) - EP,"Pop , Music , Dance , Rock , Pop/Rock , R&B / ...",$ 1.29,2011 Interscope Records,4:52,22-Nov-11,N.Y. State Of Mind ( Explicit Album Version ),Calvin Harris feat . HAIM,Pre-game Throwback,Dance & Electronic,$ 0.99,"( C ) 2015 Third Pardee Records , LLC under ex...",4:52,NaN
13,0.159220,Good To Be Alive ( Hallelujah ),Andy Grammer,Good To Be Alive ( Hallelujah ) - Single,"Pop , Music",$ 1.29,2015 S-Curve Records,3:09,24-Aug-15,Wild Child ( [ with Grace Potter ] ),Kenny Chesney,The Big Revival,Country,$ 1.29,2008 dirtybird,3:09,"September 22 , 2014"
23,0.375854,"You 're No Good ( feat . Santigold , Vybz Kart...",Major Lazer,Free the Universe ( Extended Version ),"Electronic , Music , Rock , Reggae , Modern Da...",$ 1.29,‰ ãÑ 2013 Secretly Canadian / Mad Decent,4:03,18-Jun-13,Betrayal,Brian Tyler,Thor : The Dark World ( Original Motion Pictur...,Soundtracks,$ 1.29,( C ) 2013 MARVEL,4:03,"November 5 , 2013"


## Walmart Amazon test

In [8]:
tableA.head(1)

,id,title,category,brand,modelno,price
0,0,draper infrared remote transmitter,electronics - general,draper,121066,58.45


In [5]:
tableA = pd.read_csv('Structured/Walmart-Amazon/tableA.csv')
tableB = pd.read_csv('Structured/Walmart-Amazon/tableB.csv')

In [14]:
newSamples = createPossibleMatchings(tableA,tableB,['modelno','title','brand'],0.4,50)

In [15]:
newSamples_ids = pd.DataFrame(data=newSamples,columns=['ltable_id','rtable_id'])
newSamples_ids.to_csv('Structured/Walmart-Amazon/unlabeled_ids.csv',index=False)
newSamples_df = generate_unlabeled('Structured/Walmart-Amazon/','unlabeled_ids.csv')

In [16]:
newSamples_df.head(20)

,ltable_title,ltable_category,ltable_brand,ltable_modelno,ltable_price,rtable_title,rtable_category,rtable_brand,rtable_modelno,rtable_price,id
0,startech 3-port 2b 1a 1394 mini pci express fi...,electronics - general,startech,mpex1394b3,69.00,startech.com 3 port 2b 1a 1394 mini pci expres...,firewire port cards,startech,mpex1394b3,61.24,0
1,toshiba 55 class lcd 1080p 120hz hdtv 55g310u,television,toshiba,55g310u,899.00,toshiba 55g310u 55-inch 1080p 120 hz lcd hdtv ...,tvs,toshiba,55g310u,NaN,1
2,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech webcam c260,webcams,logitech,960-000581,34.99,2
3,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech 720p webcam c310,webcams,logitech,960-000585,24.99,3
4,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech hd webcam c270,webcams,logitech,960-000694,38.35,4
5,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech 720p webcam c905,webcams,logitech,960-000578,65.01,5
6,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech webcam c210,webcams,logitech,960-000617,20.62,6
7,logitech hd 720p webcam c310,electronics - general,logitech,960-000585,48.88,logitech 720p webcam c510,computer accessories,logitech,960-000593,49.99,7
8,startech hdmi to mini hdmi cable adapter,electronics - general,startech,hdmiacfm,14.98,hdmi cable adapter hdmi to mini hdmi cable ada...,hdmi cables,startech,hdmiacfm,4.74,8
9,msi ati radeon hd6670 1gb ddr5 pci-express vid...,electronics - general,msi,r6670-md1gd5,107.00,msi ati radeon hd6670 1 gb ddr5 vga dvi hdmi p...,graphics cards,msi,r6670-md1gd5,99.99,9
